# 1 . Importing Packages 

In [4]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import nnunet
import SimpleITK
import graphviz



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [5]:
folder_dir = os.getcwd()
mount_dir = os.path.join(folder_dir, "nnUNet_Fine-tuning")
base_dir=os.getcwd()
mount_dir

'/scratch/aoueslati/nnUNet_Fine-tuning'

# 2 . Set environment Variables

In [6]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

In [7]:
print("Current Working Directory {}".format(folder_dir))
path_dict = {
    "nnUNet_raw_data_base" : os.path.join(mount_dir, "nnUNet_raw_data_base"), 
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), 
    "RESULTS_FOLDER" : os.path.join(mount_dir, "nnUNet_Results_Folder"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), 
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
    if os.getenv(env_var) != path:
        print("Error:")
        print("Environment Variable {} is not set correctly!".format(env_var))
        print("Should be {}".format(path))
        print("Variable is {}".format(os.getenv(env_var)))
    make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /scratch/aoueslati
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_raw_data_base exists.
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_preprocessed exists.
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_Results_Folder exists.
/scratch/aoueslati/nnUNet_Fine-tuning/RawData exists.
If No Error Occured Continue Forward. =)


In [8]:
# Create Folderstructure for the new task!
task_name = 'Task105_DHCP_RIB_MASKED' #change here for different task name
nnunet_raw_data = os.path.join(os.getenv("nnUNet_raw_data_base"), "nnUNet_raw_data")
task_folder_name = os.path.join(nnunet_raw_data,task_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')
main_dir = os.path.join(base_dir,'nnUNet/nnunet')

# Create Folder Structure for the DHCP_RIB_MASKED Task on the system
make_if_dont_exist(task_folder_name)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir)

training_data_name="ground_truth"
test_data_name="t2"

/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_raw_data_base/nnUNet_raw_data/Task105_DHCP_RIB_MASKED exists.
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_raw_data_base/nnUNet_raw_data/Task105_DHCP_RIB_MASKED/imagesTr exists.
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_raw_data_base/nnUNet_raw_data/Task105_DHCP_RIB_MASKED/labelsTr exists.
/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_raw_data_base/nnUNet_raw_data/Task105_DHCP_RIB_MASKED/imagesTs exists.


In [35]:
os.chdir(task_folder_name)
# download training data
!wget https://amubox.univ-amu.fr/s/ETWiAynGTFeB9xg/download/ground_truth.zip
os.chdir(base_dir)

--2022-06-13 16:13:09--  https://amubox.univ-amu.fr/s/ETWiAynGTFeB9xg/download/ground_truth.zip
Resolving amubox.univ-amu.fr (amubox.univ-amu.fr)... 139.124.245.127
Connecting to amubox.univ-amu.fr (amubox.univ-amu.fr)|139.124.245.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: 'ground_truth.zip.1'

ground_truth.zip.1      [         <=>        ]  46.22M  14.8MB/s    in 3.1s    

2022-06-13 16:13:13 (14.8 MB/s) - 'ground_truth.zip.1' saved [48470194]



In [36]:
# continue from here again.
if os.path.isfile(os.path.join(task_folder_name, training_data_name+'.zip')) is False: 
  print("Please download the dataset zipfiles and place them into the following directory: \n {}".format(task_folder_name))
else:
  print(f'file exist')

file exist


In [9]:
# verify that files are in the correct place!
os.chdir(task_folder_name)
!ls
if os.path.isfile(training_data_name+'.zip'):
    print(f'Training file exists')
else:
    print('Training file  is not present in the directory')
    print("Please check whether {}.zip is in Folder {}".format(training_data_name, task_folder_name))

os.chdir(base_dir)
print("We are currently in working directory {}".format(os.getcwd()))

dataset.json  ground_truth.zip	  imagesTr  labelsTr
ground_truth  ground_truth.zip.1  imagesTs  nnunet_105.zip
Training file exists
We are currently in working directory /scratch/aoueslati


In [9]:
#unzipping in nnUNet_raw folder the training data
os.chdir(task_folder_name)
!unzip ground_truth.zip
os.chdir(base_dir)

Archive:  ground_truth.zip
   creating: ground_truth/
   creating: ground_truth/manual_seg/
 extracting: ground_truth/manual_seg/sub-0307_ses-0369_haste_t2_masked.nii.gz  
 extracting: ground_truth/manual_seg/sub-0427_ses-0517_haste_t2_masked.nii.gz  
 extracting: ground_truth/manual_seg/sub-0457_ses-0549_haste_t2_masked.nii.gz  
 extracting: ground_truth/manual_seg/sub-0483_ses-0589_haste_t2_masked.nii.gz  
 extracting: ground_truth/manual_seg/sub-0665_ses-0791_haste_t2_masked.nii.gz  
   creating: ground_truth/t2/
 extracting: ground_truth/t2/sub-0307_ses-0369_haste_t2_masked.nii.gz  
 extracting: ground_truth/t2/sub-0427_ses-0517_haste_t2_masked.nii.gz  
 extracting: ground_truth/t2/sub-0457_ses-0549_haste_t2_masked.nii.gz  
 extracting: ground_truth/t2/sub-0483_ses-0589_haste_t2_masked.nii.gz  
 extracting: ground_truth/t2/sub-0665_ses-0791_haste_t2_masked.nii.gz  


In [10]:
# function for copying, savind and renaming
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):

    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))

In [11]:
# putting training images into folder

mask_count2 = 1 # change if more mask is available
base_data_folder_name2 = os.path.join(task_folder_name, 'ground_truth/manual_seg')



for file in os.listdir(base_data_folder_name2):
    # print(file)
    if file.endswith('.nii.gz'):
        if file.find('mask')!=-1:
            # putting mask
            shutil.move(os.path.join(base_data_folder_name2,file),train_label_dir)
        else:
            # making 4 copies
            for mask in range(1,mask_count2+1):
                new_filename2 = file[:file.find('-image')] + '-mask-r' + str(mask) + '.nii.gz'
                if mask==mask_count2:
                    copy_and_rename(base_data_folder_name2,file,train_label_dir,new_filename2,delete_original = True)
                else:
                    copy_and_rename(base_data_folder_name2,file,train_label_dir,new_filename2)
    # removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(base_data_folder_name2,file))



mask_count1 = 1 # change if more mask is available
base_data_folder_name1 = os.path.join(task_folder_name, 'ground_truth/t2')

for file in os.listdir(base_data_folder_name1):
    # print(file)
    if file.endswith('.nii.gz'):
        if file.find('mask')!=-1:
            # putting mask
            shutil.move(os.path.join(base_data_folder_name1,file),train_image_dir)
        else:
            # making 4 copies
            for mask in range(1,mask_count1+1):
                new_filename1 = file[:file.find('-image')] + '-mask-r' + str(mask) + '.nii.gz'
                if mask==mask_count1:
                    copy_and_rename(base_data_folder_name1,file,train_image_dir,new_filename1,delete_original = True)
                else:
                    copy_and_rename(base_data_folder_name1,file,train_image_dir,new_filename1)
    # removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(base_data_folder_name1,file))
        


In [12]:
def check_modality(filename):
    """
    check for the existence of modality
    return False if modality is not found else True
    """
    end = filename.find('.nii.gz')
    modality = filename[end-4:end]
    for mod in modality: 
        if not(ord(mod)>=48 and ord(mod)<=57): #if not in 0 to 9 digits
            return False
    return True

def rename_for_single_modality(directory):
    
    for file in os.listdir(directory):
        
        if check_modality(file)==False:
            new_name = file[:file.find('.nii.gz')]+"_0000.nii.gz"
            os.rename(os.path.join(directory,file),os.path.join(directory,new_name))
            print(f"Renamed to {new_name}")
        else:
            print(f"Modality present: {file}")

rename_for_single_modality(train_image_dir)
#rename_for_single_modality(train_label_dir)

Modality present: sub-0457_ses-0549_haste_t2_masked_0000.nii.gz
Modality present: sub-0483_ses-0589_haste_t2_masked_0000.nii.gz
Modality present: sub-0307_ses-0369_haste_t2_masked_0000.nii.gz
Modality present: sub-0665_ses-0791_haste_t2_masked_0000.nii.gz
Modality present: sub-0427_ses-0517_haste_t2_masked_0000.nii.gz


In [13]:
train_files = os.listdir(train_image_dir)
label_files = os.listdir(train_label_dir)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))

#assert len(set(train_files).intersection(set(label_files))) == 5 #should be equal to 160 for SCGM Challenge

train image files: 5
train label files: 5
Matches: 0


# Model

In [14]:
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:

    json_dict = OrderedDict()
    json_dict['name'] = task_name
    json_dict['description'] = "MRI Segmentation"
    json_dict['tensorImageSize'] = "3D"
    json_dict['reference'] = "INT"
    json_dict['licence'] = "nnUNET"
    json_dict['release'] = "0.0"

    #you may mention more than one modality
    json_dict['modality'] = {
        "0": "MRI"
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "0": "a",
        "1": "b",
        "2": "c",
        "3": "d",
        "4": "e",
        "5": "f",
        "6": "g",
        "7": "h",
        "8": "i",
        "9": "j"
    }
    
    train_ids = os.listdir(train_label_dir)
    test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTest'] = len(test_ids)

    #no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    #removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % (i[:i.find("_0000")]+'.nii.gz') for i in test_ids]

    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json already exist!
dataset.json overwritten!


# Import the model

In [17]:
os.chdir(base_dir)
# download training data
!wget https://amubox.univ-amu.fr/s/8Wpazd9y52cJXEZ/download/nnunet_105.zip

--2022-06-09 14:49:11--  https://amubox.univ-amu.fr/s/8Wpazd9y52cJXEZ/download/nnunet_105.zip
Resolving amubox.univ-amu.fr (amubox.univ-amu.fr)... 139.124.245.127
Connecting to amubox.univ-amu.fr (amubox.univ-amu.fr)|139.124.245.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1147990500 (1.1G) [application/zip]
Saving to: 'nnunet_105.zip'

nnunet_105.zip      100%[===================>]   1.07G   203MB/s    in 5.2s    

2022-06-09 14:49:17 (209 MB/s) - 'nnunet_105.zip' saved [1147990500/1147990500]



# Install the model

In [15]:
!nnUNet_install_pretrained_model_from_zip nnunet_105.zip



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



# Preprocess 

In [27]:
import torch
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
from nnunet.paths import preprocessing_output_dir
join(preprocessing_output_dir, task_name)

'/scratch/aoueslati/nnUNet_Fine-tuning/nnUNet_preprocessed/Task105_DHCP_RIB_MASKED'

In [29]:
task_name = 'Task105_DHCP_RIB_MASKED'
plans_fname = join(preprocessing_output_dir, task_name, 'nnUNetPlansv2.1_plans_3D.pkl')
plans = load_pickle(plans_fname)
plans['plans_per_stage'][0]['batch_size'] = 2
plans['plans_per_stage'][0]['median_patient_size_in_voxels'] = np.array([160, 196, 174])
plans['plans_per_stage'][0]['current_spacing'] = np.array([0.5, 0.5, 0.5])
plans['plans_per_stage'][0]['patch_size'] = np.array([128, 128, 128])
plans['plans_per_stage'][0]['num_pool_per_axis'] = [5, 5, 5]
plans['plans_per_stage'][0]['do_dummy_2D_data_aug'] = False
plans['plans_per_stage'][0]['original_spacing'] = np.array([0.5, 0.5, 0.5])
plans['plans_per_stage'][0]['pool_op_kernel_sizes'] = [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]]
plans['plans_per_stage'][0]['conv_kernel_sizes'] = [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
save_pickle(plans, join(preprocessing_output_dir, task_name, 'NewnnUNetPlansv2.1_plans_3D.pkl'))

In [33]:
!nnUNet_plan_and_preprocess -t 105 -overwrite_plans NewnnUNetPlansv2.1_plans_3D.pkl -pl3d ExperimentPlanner3D_v21_Pretrained -overwrite_plans_identifier NewnnUNetPlansv2.1_plans_3D.pkl



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Overwriting plans only works for the 3d planner. I am setting '--planner2d' to None. This will skip 2d planning and preprocessing.
sub-0665_ses-0791_haste_t2_masked
sub-0427_ses-0517_haste_t2_masked
sub-0483_ses-0589_haste_t2_masked
sub-0307_ses-0369_haste_t2_masked
sub-0457_ses-0549_haste_t2_masked



 Task105_DHCP_RIB_MASKED
number of threads:  (8, 8) 

using nonzero mask for normalization
Are we using the nonzero mask for normalization? OrderedDict([(0, True)])
the median shape of the dataset is  [160. 196. 174.]
the max shape in the dataset is  [174. 214. 179.]
the min shape in the dataset is  [155. 185. 159.]
we don't want fe

In [26]:
!nnUNet_plan_and_preprocess -t 105 -overwrite_plans nnUNetPlansv2.1_plans_3D.pkl -pl3d ExperimentPlanner3D_v21_Pretrained -overwrite_plans_identifier Alex



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Overwriting plans only works for the 3d planner. I am setting '--planner2d' to None. This will skip 2d planning and preprocessing.
sub-0665_ses-0791_haste_t2_masked
sub-0427_ses-0517_haste_t2_masked
sub-0483_ses-0589_haste_t2_masked
sub-0307_ses-0369_haste_t2_masked
sub-0457_ses-0549_haste_t2_masked



 Task105_DHCP_RIB_MASKED
number of threads:  (8, 8) 

using nonzero mask for normalization
Are we using the nonzero mask for normalization? OrderedDict([(0, True)])
the median shape of the dataset is  [160. 196. 174.]
the max shape in the dataset is  [174. 214. 179.]
the min shape in the dataset is  [155. 185. 159.]
we don't want fe

# A REVOIR 

In [54]:
#!nnUNet_plan_and_preprocess -t 105 -overwrite_plans pretrained_weights/new_model.model -pl3d ExperimentPlanner3D_v21_Pretrained -overwrite_plans_identifier pretrained_weights/new_model.model

In [1]:
#!nnUNet_train 3d_fullres nnUNetTrainerV2 Task105_DHCP_RIB_MASKED 0 -w pretrained_weights/new_model.model

In [5]:
#!sbatch finetun1.sh

In [24]:
#!nnUNet_determine_postprocessing -m 3d_fullres -t 105 -tr nnUNetTrainerV2  -val validation_raw

In [6]:
#!nano nnunet-7734648_3d_full_res.out

In [30]:
#from PIL import Image

In [29]:
#from IPython.display import Image
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNET_folder/nnUNet_Results_Folder/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/progress.png')

In [27]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNET_folder/nnUNet_Results_Folder/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV2__nnUNetPlansv2.1/fold_1/progress.png')

In [26]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNET_folder/nnUNet_Results_Folder/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV2__nnUNetPlansv2.1/fold_2/progress.png')

In [25]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNet_folder/nnUNet_trained_models/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV3__nnUNetPlansv2.1/fold_0/progress.png')

In [24]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNet_folder/nnUNet_trained_models/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV3__nnUNetPlansv2.1/fold_1/progress.png')

In [23]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNet_folder/nnUNet_trained_models/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV3__nnUNetPlansv2.1/fold_3/progress.png')

In [22]:
#Image('/scratch/aoueslati/scratch/aoueslati/nnUNet_folder/nnUNet_trained_models/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV3__nnUNetPlansv2.1/fold_4/progress.png')

In [21]:
#C-t 105 -overwrite_plans ExperimentPlanner3D_v21_Pretrained

In [20]:
#!nnUNet_train 3d_fullres nnUNetTrainerV3 Task105_DHCP_RIB_MASKED 0 

In [19]:
#!nnUNet_determine_postprocessing -t 105 -m 3d_fullres -pl nnUNetTrainerV3__nnUNetPlansv2.1

In [19]:
#!nnUNet_predict -i /scratch/aoueslati/predict -o /scratch/aoueslati/scratch/aoueslati/nnUNet_folder/nnUNet_Results_Folder/nnUNet/3d_fullres/Task105_DHCP_RIB_MASKED/nnUNetTrainerV3__nnUNetPlansv2.1 -f 0 -t Task105_DHCP_RIB_MASKED -tr nnUNetTrainerV3